In [ ]:
from tkinter import messagebox from tkinter import *
from tkinter import simpledialog import tkinter
from tkinter import filedialog
from tkinter.filedialog import askopenfilename import numpy as np
import matplotlib.pyplot as plt import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split import os
from sklearn.metrics import confusion_matrix from sklearn.metrics import accuracy_score from sklearn.naive_bayes import GaussianNB
 
from sklearn.metrics import precision_score from sklearn.metrics import recall_score from sklearn.metrics import f1_score import seaborn as sns
from sklearn.preprocessing import LabelEncoder from sklearn.preprocessing import normalize from imblearn.over_sampling import SMOTE import pickle
global filename, le global X,Y
global dataset global main global text accuracy = [] precision = [] recall = [] fscore = []
global X_train, X_test, y_train, y_test, predict_cls global classifier
main = tkinter.Tk()
main.title("Enhanced CyberCrime Underground Economy Using Machine Learning Approaches") #designing main screen
main.geometry("1300x1200")
labels = ['Benign', 'Bot', 'Brute Force -Web', 'Brute Force -XSS', 'Infilteration', 'SQL Injection'] #fucntion to upload dataset
def uploadDataset(): global filename global dataset
text.delete('1.0', END)
filename = filedialog.askopenfilename(initialdir="Dataset") text.insert(END,filename+" loaded\n\n")
dataset = pd.read_csv(filename) #replace missing values with 0
 
dataset.fillna(0, inplace = True) text.insert(END,"Dataset before preprocessing\n\n") text.insert(END,str(dataset.head())+"\n\n")
unique, count = np.unique(dataset['Label'], return_counts=True) for i in range(len(unique)):
text.insert(END,str(unique[i])+" = "+str(count[i])+"\n") text.update_idletasks()
label = dataset.groupby('Label').size() label.plot(kind="bar") plt.xlabel('Various Cybercrime Attacks') plt.ylabel('Count')
plt.title('Cybercrime Graph') plt.show()
#function to perform dataset preprocessing def analyticalProcessing():
global X, Y, le global dataset
global X_train, X_test, y_train, y_test text.delete('1.0', END)
dataset.drop(columns=['Timestamp','Flow Byts/s','Flow Pkts/s'],inplace=True) le = LabelEncoder()
dataset['Label'] = pd.Series(le.fit_transform(dataset['Label'].astype(str))) dataset = dataset.values
X = dataset[:,0:dataset.shape[1]-1] Y = dataset[:,dataset.shape[1]-1] indices = np.arange(X.shape[0]) np.random.shuffle(indices)
X = X[indices] Y = Y[indices]
sm = SMOTE(random_state=42)
smote_X, smote_Y = sm.fit_resample(X, Y)
X_train, X_test, y_train, y_test = train_test_split(smote_X, smote_Y, test_size=0.2,
 
random_state=0) print(Y) print(X)
text.insert(END,"Dataset after Preprocessing\n\n") text.insert(END,str(X)+"\n\n")
text.insert(END,"Total records found in dataset : "+str(X.shape[0])+"\n") text.insert(END,"Total features found in dataset: "+str(X.shape[1])+"\n\n") text.insert(END,"Dataset Train and Test Split\n\n")
text.insert(END,"80% dataset records used to train Naive Bayes algorithms : "+str(X_train.shape[0])+"\n")
text.insert(END,"20% dataset records used to train Naive Bayes algorithms : "+str(X_test.shape[0])+"\n")
def calculateMetrics(algorithm, predict, y_test): a = accuracy_score(y_test,predict)*100
p = precision_score(y_test, predict,average='macro') * 100 r = recall_score(y_test, predict,average='macro') * 100
f = f1_score(y_test, predict,average='macro') * 100 accuracy.append(a)
precision.append(p) recall.append(r) fscore.append(f)
text.insert(END,algorithm+" Accuracy : "+str(a)+"\n") text.insert(END,algorithm+" Precision : "+str(p)+"\n") text.insert(END,algorithm+" Recall	: "+str(r)+"\n") text.insert(END,algorithm+" FScore		: "+str(f)+"\n\n")
def runNaiveBayes():
global X,Y, X_train, X_test, y_train, y_test, classifier global accuracy, precision,recall, fscore accuracy.clear()
precision.clear() recall.clear() fscore.clear() text.delete('1.0', END)
 
if os.path.exists('model/nb.txt'):
with open('model/nb.txt', 'rb') as file:
classifier = pickle.load(file) file.close()
else:
classifier = GaussianNB() classifier.fit(X, Y)
with open('model/nb.txt', 'wb') as file: pickle.dump(classifier, file) file.close()
predict = classifier.predict(X_test) calculateMetrics("Naive Bayes", predict, y_test)
def predict(): global classifier
text.delete('1.0', END)
filename = filedialog.askopenfilename(initialdir="Dataset") text.insert(END,filename+" loaded\n\n")
dataset = pd.read_csv(filename,encoding='iso-8859-1') dataset.fillna(0, inplace = True)
dataset.drop(columns=['Timestamp','Flow Byts/s','Flow Pkts/s'],inplace=True) dataset = dataset.values
prediction = classifier.predict(dataset) print(prediction)
for i in range(len(prediction)):
text.insert(END,"Test DATA : "+str(dataset[i])+" ===> PREDICTED AS "+labels[int(prediction[i])]+"\n\n")
def graph():

df = pd.DataFrame([['Naive Bayes','Precision',precision[0]],['Naive Bayes','Recall',recall[0]],['Naive Bayes','F1 Score',fscore[0]],['Naive Bayes','Accuracy',accuracy[0]], columns=['Algorithms','Performance Output','Value'])
df.pivot("Algorithms", "Performance Output", "Value").plot(kind='bar')
 
plt.show() def close():
main.destroy()
font = ('times', 16, 'bold')
title = Label(main, text='Enhanced CyberCrime Underground Economy Using Machine Learning Approaches')
title.config(bg='greenyellow', fg='dodger blue') title.config(font=font)
title.config(height=3, width=120) title.place(x=0,y=5)
font1 = ('times', 12, 'bold') text=Text(main,height=20,width=150) scroll=Scrollbar(text) text.configure(yscrollcommand=scroll.set) text.place(x=50,y=120) text.config(font=font1)
font1 = ('times', 13, 'bold')
uploadButton = Button(main, text="Dataset Upload & Analysis", command=uploadDataset) uploadButton.place(x=50,y=550)
uploadButton.config(font=font1)
processButton = Button(main, text="Dataset Processing & Analytical Methods", command=analyticalProcessing)
processButton.place(x=370,y=550) processButton.config(font=font1)
nbButton = Button(main, text="Run Naive Bayes Classification Model", command=runNaiveBayes)
nbButton.place(x=750,y=550) nbButton.config(font=font1)
graphButton = Button(main, text="Classification Performance Graph", command=graph) graphButton.place(x=50,y=600)
graphButton.config(font=font1)
predictButton = Button(main, text="Predict Cyber Crime", command=predict) predictButton.place(x=370,y=600)
predictButton.config(font=font1)
 
closeButton = Button(main, text="Exit", command=close) closeButton.place(x=750,y=600) closeButton.config(font=font1) main.config(bg='LightSkyBlue')
main.mainloop()
